<a href="https://www.kaggle.com/code/wasswassss/xgboost-on-home-data?scriptVersionId=119539328" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
    
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/home-data-for-ml-course/train.csv


# Importing data / Specifying features and target

In [2]:
from sklearn.model_selection import train_test_split

#importing the database
data=pd.read_csv('/kaggle/input/home-data-for-ml-course/train.csv')

#droping rows with missing target value
data.dropna(axis=0, subset=['SalePrice'],inplace=True)

#Defining the target
y=data.SalePrice
#Defining the features + Dropping columns with more than 80% of NA Values
data.drop(['SalePrice','Alley','Fence','MiscFeature','PoolQC'],axis=1,inplace=True)



# Splitting the data and encoding categorical values

In [3]:

X_train, X_valid, y_train, y_valid = train_test_split(data, y, train_size=0.8, test_size=0.2,random_state=0)

#Encoding Categorical Values
X_train = pd.get_dummies(X_train)
X_valid = pd.get_dummies(X_valid)
X_train, X_valid = X_train.align(X_valid, join='left', axis=1)

#A glimpse of the training data we're working with
X_train.head()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
618,619,20,90.0,11694,9,5,2007,2007,452.0,48,...,0,1,0,0,0,0,0,0,0,1
870,871,20,60.0,6600,5,5,1962,1962,0.0,0,...,0,0,0,1,0,0,0,0,1,0
92,93,30,80.0,13360,5,7,1921,2006,0.0,713,...,0,0,0,1,0,0,0,0,1,0
817,818,20,NaN,13265,8,5,2002,2002,148.0,1218,...,0,0,0,1,0,0,0,0,1,0
302,303,20,118.0,13704,7,5,2001,2002,150.0,0,...,0,0,0,1,0,0,0,0,1,0


In [4]:
numerical_cols = [cname for cname in X_train.columns if X_train[cname].dtype in ['int64', 'float64']]
print(numerical_cols)

['Id', 'MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold']


# Handling missing values with a SimpleImputer

In [5]:

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

#Since all the values are now numerical, we only need a numerical transformer
#I used the median strategy (choice)
numerical_transformer = SimpleImputer(strategy='median')



preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols)
    ])
X_train.shape

(1168, 272)

# Creating the model

In [6]:
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

model = XGBRegressor(n_estimators=400, learning_rate=0.1, n_jobs=4)

In [7]:
#Preprocessing and fitting
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                             ])

my_pipeline.fit(X_train,y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  SimpleImputer(strategy='median'),
                                                  ['Id', 'MSSubClass',
                                                   'LotFrontage', 'LotArea',
                                                   'OverallQual', 'OverallCond',
                                                   'YearBuilt', 'YearRemodAdd',
                                                   'MasVnrArea', 'BsmtFinSF1',
                                                   'BsmtFinSF2', 'BsmtUnfSF',
                                                   'TotalBsmtSF', '1stFlrSF',
                                                   '2ndFlrSF', 'LowQualFinSF',
                                                   'GrLivArea', 'BsmtFullBath',
                                                   'BsmtHalfBath', 'Ful...
                              enable_categorical

# Testing model's perf

#### Computing MAE

In [8]:
from sklearn.metrics import mean_absolute_error
# Preprocessing of validation data, get predictions
preds = my_pipeline.predict(X_valid)
# Evaluating the model
score = mean_absolute_error(y_valid, preds)
print('MAE:', score)


MAE: 17130.14834385702


#### Cross-Validation

In [9]:
from sklearn.model_selection import cross_val_score

scores = -1 * cross_val_score(my_pipeline, data, y,
                              cv=5,
                              scoring='neg_mean_absolute_error')
print("MAE scores:\n", scores)
print("Average MAE :\n", scores.mean())

MAE scores:
 [16735.19556667 17218.2627622  17535.35667273 14873.08128211
 18388.02918985]
Average MAE :
 16949.985094713185
